## 2. 크롤링 (문법 정리), method를 사용하여 표현해 주세요.

### 구글 플레이 스토어에서 앱 검색 후 댓글을 수집하는 코드를 작성하시오.

- 조건1: 내가 입력한 검색어를 기반으로 첫 번째 앱을 선택합니다.
- 조건2: 댓글 수집이 100개 이상 수집되어야 합니다. (댓글 갯수가 100미만인 앱은 예외)
- 조건3: 댓글에서 아래의 정보를 가져오세요.
    - 작성자명, 별표 갯수, 작성일, 작성 내용
- 조건4: 크롤링한 데이터를 오늘 날짜를 기준으로 엑셀에 저장하시오. (파일명: 2023-04-18)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [36]:
# (1) 링크 생성
keyword = input("수집하고자 하는 앱 이름을 입력하세요 : ")
search_url = f'https://play.google.com/store/search?q={keyword}&c=apps&hl=en-KR'

print(search_url)

https://play.google.com/store/search?q=크림&c=apps&hl=en-KR


In [37]:
# (2) 브라우저 열고 페이지 이동
browser = webdriver.Chrome()
browser.get(search_url)

In [38]:
# (3) 앱의 상세 페이지로 이동
# 첫 번째 앱 선택
# ## class="Qfxief"
### url로 바로 이동하기
app_url = browser.find_element(By.CLASS_NAME, 'Qfxief').get_attribute('href')
browser.get(app_url)

In [ ]:
### click으로 이동하기
## class="VfPpkd-EScbFb-JIbuQc TAQqTe"
#browser.find_element(By.CLASS_NAME, 'VfPpkd-EScbFb-JIbuQc').click()

In [ ]:
# 2.
## (4) [리뷰 모두 보기] 버튼 누르기
## //*[@id="yDmH0d"]/c-wiz[3]/div/div/div[1]/div/div[2]/div/div[1]/div[1]/c-wiz[5]/section/div/div[2]/div[5]/div/div/button
browser.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[3]/div/div/div[1]/div/div[2]/div/div[1]/div[1]/c-wiz[5]/section/div/div[2]/div[5]/div/div/button').click()
##FIXME: xpath가 앱마다 @id 부분이 달라서 일관된 적용을 하려면 개선이 필요함

In [41]:
# (5) 리뷰 스크롤 >> JS
## 모달 내부 클릭
modal_elem = browser.find_element(By.CLASS_NAME, 'fysCi')
modal_elem.click()

In [45]:
##FIXME: 어떻게 끝까지 스크롤 해야 할까
# https://developer.mozilla.org/ko/docs/Web/API/Window/scrollTo
## execute_script('js'): js 코드 실행
## 댓글을 수집하기 위해 3번 스크롤
for i in range(3):
    js_code = 'arguments[0].scrollTo(0, arguments[0].scrollHeight)'
    browser.execute_script(js_code, modal_elem)
    time.sleep(1)

In [60]:
# 댓글 컨테이너:
## class="RHo1pe"
datas = browser.find_elements(By.CLASS_NAME, 'RHo1pe')
data_list = []

for data in datas:
    ## 작성자명
    ## class="X5PpBb"
    username = data.find_element(By.CLASS_NAME, 'X5PpBb').text

    ## 별표 갯수
    ## class="iXRFPc"
    ## aria-label="Rated 1 stars out of five stars"
    # rank = data.find_element(By.CLASS_NAME, 'iXRFPc').get_attribute("aria-label")[6:7]
    ## aria-label="별표 5개 만점에 1개를 받았습니다."
    rank = data.find_element(By.CLASS_NAME, 'iXRFPc').get_attribute("aria-label")[10:11]
    
    ## 작성일
    ## class="bp9Aid"
    date = data.find_element(By.CLASS_NAME, 'bp9Aid').text
    # print("comment_date: ", comment_date)

    ## 작성 내용
    ## class="h3YV2d"
    content = data.find_element(By.CLASS_NAME, 'h3YV2d').text
    # print("content: ", content)

    data_list.append({
        '유저명': username,
        '별표': rank,
        '작성일': date,
        '작성 내용': content
    })


print(data_list)


[{'유저명': '박지은', '별표': '1', '작성일': '2024년 3월 22일', '작성 내용': '왜이렇게 업데이트 할때마다 사용하기 점점 불편하게 바뀌는지? 클릭한번 더 해야되고 더 번거롭게 바뀌네요. 일반결제가 기본임에도 저쪽 구석으로 밀려나고 마치 한집배달을 유도하듯 메인에 만들어 놔서 잘못 클릭하면 자꾸 나가서 일반결제를 다시 클릭해야하게 만들어놓더니. 이젠 결제창에 요청사항이나 리뷰이벤트할때 또 클릭해서 하게끔 바뀌었네요. 쓸데없는 짓 정말 잘하네요 배민.'}, {'유저명': 'kh ki', '별표': '1', '작성일': '2024년 4월 5일', '작성 내용': 'UI 바뀌고 진짜 개햇갈리게 바꿔놨네요.. 모던도르에 빠져서 편의성이나 시인성이 거지같아진건 아닌지 한번 생각해봤으면 좋겠습니다 그리고 일회용품 체크되서 나오는거 개편하면서 꽁꽁감싸숨겨둬서 배달할때마다 전화해야하는거 개열받네요 진짜.. 이게 목적이셨다면 성공하셨습니다 축하드립니다'}, {'유저명': '니가가라카와이', '별표': '1', '작성일': '2024년 4월 5일', '작성 내용': '삭제하기 전 참고해서 변화가 있길 바라며 글 남기고 갑니다. 이용하는 동안 정말 편하게 썼습니다. 어플 편의성으로만 보면 별4개 정도이나, 한개주는건 얼탱이가 없어서. 3월 약 50만원을 지출하고 등급이 업데이트 되었다고 톡이 왔는데요. 발송일 4월2일, 만료일 4월7일. 한달 지속도 아니고 5일 후 만료에 어처구니가 없습니다. 쿠폰도 다양성 따윈 없고 이용빈도 낮은거 4장. 등급에 대한 별도 혜택도 없는데 등급이 왜 존재하는지 의문입니다. 많은분들이 이용해서 꽤나 규모가 큰 업체로 성장하셨는데, 그에 반해 혜택은 별 볼일 없네요. 번창하세요'}, {'유저명': '이유빈', '별표': '1', '작성일': '2024년 3월 30일', '작성 내용': '아무리 생각해봐도 배달의민족 어플리케이션 UI관련 부서 문제있어보이네요 기존에 주문창에서 한번에 볼 수 있던 UI를 포기하고 각각 메뉴 하나

In [57]:
## 데이터 프레임
df = pd.DataFrame(data_list)

In [ ]:
# 4. <- 이 부분은 그냥 저장하셨다.
df.to_csv("goolge_play_store.csv", encoding='utf-8-sig')